In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
df = pd.read_csv('./data/master.csv')

In [3]:
df.head(10)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,15,66,24,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,15,66,24,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,15,66,24,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,15,66,24,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,15,66,24,900",796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,Albania1987,NaN,"2,15,66,24,900",796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,Albania1987,NaN,"2,15,66,24,900",796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,Albania1987,NaN,"2,15,66,24,900",796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,Albania1987,NaN,"2,15,66,24,900",796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,Albania1987,NaN,"2,15,66,24,900",796,Generation X


In [4]:
df.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [5]:
cat_cols = ['country', 'year', 'sex', 'age', 'generation']
con_cols = ['suicides_no', 'population', 'suicides/100k pop', ' gdp_for_year ($) ', 'gdp_per_capita ($)']

In [6]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].apply(lambda x: int(x.replace(',', '')))

In [7]:
for cat in cat_cols:
    df[cat] = df[cat].astype('category')
    df[cat] = df[cat].cat.codes
    
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df[con_cols].values)
df[con_cols] = scaler.transform(df[con_cols])

In [8]:
df.head(10)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,0,2,1,0,-0.998120,-0.985727,-0.940348,Albania1987,NaN,-0.999767,-0.991356,2
1,0,2,1,2,-0.998567,-0.985950,-0.953861,Albania1987,NaN,-0.999767,-0.991356,5
2,0,2,0,0,-0.998747,-0.986786,-0.957061,Albania1987,NaN,-0.999767,-0.991356,2
3,0,2,1,5,-0.999910,-0.999017,-0.959195,Albania1987,NaN,-0.999767,-0.991356,1
4,0,2,1,1,-0.999194,-0.987489,-0.970841,Albania1987,NaN,-0.999767,-0.991356,0
5,0,2,0,5,-0.999910,-0.998387,-0.975019,Albania1987,NaN,-0.999767,-0.991356,1
6,0,2,0,2,-0.999463,-0.987284,-0.980886,Albania1987,NaN,-0.999767,-0.991356,5
7,0,2,0,1,-0.999642,-0.988270,-0.986131,Albania1987,NaN,-0.999767,-0.991356,0
8,0,2,1,4,-0.999910,-0.993735,-0.993510,Albania1987,NaN,-0.999767,-0.991356,1
9,0,2,0,3,-1.000000,-0.985813,-1.000000,Albania1987,NaN,-0.999767,-0.991356,2


In [9]:
df.tail(10)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
27810,100,29,0,0,-0.968932,-0.863370,-0.896964,Uzbekistan2014,0.675,-0.993044,-0.967359,4
27811,100,29,1,4,-0.987107,-0.941978,-0.899275,Uzbekistan2014,0.675,-0.993044,-0.967359,0
27812,100,29,1,0,-0.968932,-0.857248,-0.901320,Uzbekistan2014,0.675,-0.993044,-0.967359,4
27813,100,29,1,5,-0.998478,-0.989740,-0.932791,Uzbekistan2014,0.675,-0.993044,-0.967359,5
27814,100,29,0,1,-0.985496,-0.875130,-0.947371,Uzbekistan2014,0.675,-0.993044,-0.967359,4
27815,100,29,0,2,-0.990420,-0.834696,-0.973685,Uzbekistan2014,0.675,-0.993044,-0.967359,2
27816,100,29,0,5,-0.999194,-0.984103,-0.977064,Uzbekistan2014,0.675,-0.993044,-0.967359,5
27817,100,29,1,3,-0.994628,-0.873901,-0.980709,Uzbekistan2014,0.675,-0.993044,-0.967359,3
27818,100,29,0,3,-0.996061,-0.879862,-0.985154,Uzbekistan2014,0.675,-0.993044,-0.967359,3
27819,100,29,0,4,-0.998120,-0.934315,-0.987020,Uzbekistan2014,0.675,-0.993044,-0.967359,0
